In [18]:
from IPython.display import HTML
#DOESN'T SEEM T WORK
HTML("""
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
""")
#aim is to autorun all

IPython.core.display.HTML

In [3]:
from Jumpscale import j
simulation = j.tools.tfsimulation.simulation_get()
environment = simulation.environment_get("main")
from IPython.display import Markdown as md
import numpy as np
import bqplot.pyplot as plt
from bqplot import *

## environment arguments

In [4]:
environment.cost_power_kwh = "0.15 USD"
environment.cost_rack_unit = "12 USD"
environment.bandwidth_mbit = 20

# sales arguments
environment.sales_price_cu = "10 USD"
environment.sales_price_su = "6 USD"

device_edge = simulation.device_get("edge1", environment=environment)
# print(d1)

switch = simulation.device_get("switch", environment=environment)

environment.device_add("edge1", device_edge, 20)
environment.device_add("switch", switch, 2)

# print(environment)

simulation_run = simulation.run()

# month:growth_percent of nodes being added
simulation_run.growth_percent_set("3:5,11:8,24:10,36:12,48:10,60:10")

# means at end of period we produce 40% more cpr (*1.4)
# cpr = capacity production rate (is like hashrate of bitcoin)
simulation_run.cpr_improve_set("71:40")

# price of a capacity unit goes down over time, here we say it will go down 40%
# means we expect price to be lowered by X e.g. 40 (*0.6)
simulation_run.cpr_sales_price_decline_set("0:100,71:40")

# utilization of the nodes, strating with 0
simulation_run.utilization_set("30:80,71:90")

# super important factor, how does token price goes up, this is ofcourse complete speculation, no-one knows
simulation_run.tokenprice_set("0:0.15,71:2")

# are the parameters for a batch of nodes to be added
nb = simulation_run.nodes_batch_template_get(environment)
# print(nb)

# do the calculation of the simulation
simulation_run.calc(nr_start_nodes=1500, months_remaining_start_nodes=36, nodes_batch_template=nb)


In [5]:
C="""
```python
%s
```
"""%In[2]
m=md(C)
m


```python
from IPython.display import HTML
HTML('''
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
''')
```


### calculated edge device

In [6]:
device_edge

# JSXOBJ:
## threefold.simulation.device
ID: 6
 - name                : edge1
 - description         : 
 - device_template_name: edge1
 - cost                : 3,627
 - power               : 222
 - rackspace_u         : 2
 - cru                 : 16
 - sru                 : 1000
 - hru                 : 60000
 - mru                 : 128
 - su                  : 58
 - cu                  : 31
 - cu_passmark         : 771
 - su_perc             : 62.95%
 - cu_perc             : 37.05%
 - cpr                 : 105
 - cost_su_capex       : 39.14204775394577
 - cost_cu_capex       : 42.321697039994646
 - cost_power          : 23.976
 - cost_rack           : 24
 - cost_month          : 108.426
 - cost_cu_month       : 1.2651702021666555
 - cost_su_month       : 1.170117361392149


### environment of 1500 edge devices

Is a complete environment with switches and 1500 edge devices


In [7]:
environment

# JSXOBJ:
## threefold.simulation.environment
ID: 5
 - name                : main
 - description         : 
 - cost_power_kwh      : 0.15
 - cost_rack_unit      : 12
 - cost                : 81,540
 - power               : 4,640
 - rackspace_u         : 42
 - cru                 : 320
 - sru                 : 20000
 - hru                 : 1200000
 - mru                 : 2560
 - su                  : 1166
 - cu                  : 634
 - cu_passmark         : 771
 - su_perc             : 62.95%
 - cu_perc             : 37.05%
 - cpr                 : 2,118
 - cost_su_capex       : 43.998381222177244
 - cost_cu_capex       : 47.610016139942005
 - cost_power          : 501.12
 - cost_rack           : 504
 - cost_month          : 2364.12
 - cost_cu_month       : 1.380375169938186
 - cost_su_month       : 1.275661675435046
 - bandwidth_mbit      : 20
 - cost_bandwidth      : 0
 - nr_devices          : 20
 - sales_price_cu      : 10
 - sales_price_su      : 6
 - sales_price_nu      : 0
 - s

In [8]:
plt.figure(title='Growth Percent Per Month')
plt.plot(np.arange(simulation_run.sheet.nrcols), simulation_run.sheet.rows["growth_percent"].cells)
plt.show()

In [9]:
plt.figure(title='Nr Nodes')
plt.plot(np.arange(simulation_run.sheet.nrcols), simulation_run.sheet.rows["nrnodes"].cells)
plt.show()

In [10]:
plt.figure(title='Nr Racks Occupied.')
plt.plot(np.arange(simulation_run.sheet.nrcols), simulation_run.sheet.rows["nrracks"].cells)
plt.show()

In [11]:
plt.figure(title='Revenue Per Month (kUSD).')
plt.plot(np.arange(simulation_run.sheet.nrcols), [int(i/1000) for i in simulation_run.sheet.rows["revenue_month"].cells])
plt.show()

In [12]:
plt.figure(title='Recurring Revenue Over 60 Months (mUSD).')
plt.plot(np.arange(simulation_run.sheet.nrcols), [float(i*60/1000000) for i in simulation_run.sheet.rows["revenue_month"].cells])
plt.show() 

In [13]:
simulation_run.sheet.rows

{'growth_percent': ['growth_percent', 0.0, 1.67, 3.33, 5.0, 5.38, 5.75, 6.12, 6.5, 6.88, 7.25, 7.62, 8.0, 8.15, 8.31, 8.46, 8.62, 8.77, 8.92],
 'cpr_improve': ['cpr_improve', 0.0, 0.56, 1.13, 1.69, 2.25, 2.82, 3.38, 3.94, 4.51, 5.07, 5.63, 6.2, 6.76, 7.32, 7.89, 8.45, 9.01, 9.58],
 'cpr_sales_price_decline': ['cpr_sales_price_decline', 100.0, 99.15, 98.31, 97.46, 96.62, 95.77, 94.93, 94.08, 93.24, 92.39, 91.55, 90.7, 89.86, 89.01, 88.17, 87.32, 86.48, 85.63],
 'utilization': ['utilization', 0.0, 2.67, 5.33, 8.0, 10.67, 13.33, 16.0, 18.67, 21.33, 24.0, 26.67, 29.33, 32.0, 34.67, 37.33, 40.0, 42.67, 45.33],
 'tokenprice': ['tokenprice', 0.15, 0.18, 0.2, 0.23, 0.25, 0.28, 0.31, 0.33, 0.36, 0.38, 0.41, 0.44, 0.46, 0.49, 0.51, 0.54, 0.57, 0.59],
 'nrnodes_new': ['nrnodes_new', 1500, 25.05, 50.782500000000006, 78.75, 88.9314, 100.1075, 112.6692, 126.94500000000001, 143.0352, 161.095, 181.58460000000002, 205.12, 225.67350000000002, 248.80140000000003, 274.27320000000003, 303.0792, 334.9263, 3

In [19]:
[int(i) for i in simulation_run.sheet.rows["revenue_month"].cells]
#IS TOO SMALL OUTPUT NEEDS TO BE BIGGER
#TODO: need to fix


[0,
 26,
 52,
 81,
 112,
 147,
 185,
 227,
 275,
 328,
 389,
 457,
 535,
 623,
 722,
 835,
 962,
 1105,
 1267,
 1451,
 1659,
 1894,
 2162,
 2466,
 2811,
 3204,
 3649,
 4157,
 4734,
 5393,
 6149,
 6802,
 7541,
 8363,
 9295,
 10345,
 11524,
 12860,
 14317,
 15917,
 17671,
 19569,
 21648,
 23886,
 26322,
 28964,
 31801,
 34861,
 38136,
 41648,
 45469,
 49632,
 54195,
 59120,
 64509,
 70337,
 76702,
 83637,
 91141,
 99300,
 108145,
 117740,
 128207,
 139485,
 151761,
 164973,
 179342,
 194895,
 211696,
 229833,
 249404,
 270502]

In [15]:
270000*60

16200000